# Exploring OPTAA sensor data
In preparation for an OOIFB summer school in July 2023, I am developing a baseline knowledge about the spectrophotometer data that is available from OOI. The week-long summer school will educate OOI data users on how to access OPTAA (Spectrophotometer) instrument data, prepare it for analysis, and where the data comes from.

I am requesting and downloading data from one of the Coastal Pioneer NES array nodes that had an OPTAA instrument on a more recent deployment. After checking the content and structure of the downloaded dataset, I will plot a couple of different views of the data. I am interested in whether annotated data is already removed from the dataset, and how the visualizations I create will compare to what is currently available at [OOI Data Explorer](https:\\dataexplorer.oceanobservatories.org).

In [1]:
# Import modules needed for this notebook

import os
import ooinet
import numpy as np
import pandas as pd
import xarray as xr
import dask
import matplotlib.pyplot as plt

from ooi_data_explorations import common

# eventually when I have source code in this project folder, I will include the following line:
# import optaa_data_explorations as optaa

In [2]:
# Pick reference designator for instrument of interest
refdes = 'CP03ISSM-RID27-01-OPTAAD000'

[site, node, sensor] = refdes.split('-', 2)

In [3]:
# Show available data recovery methods for refdes (maybe also show metadata)
methods = common.list_methods(site, node, sensor)
methods

['recovered_host', 'telemetered']

In [4]:
# Show available data streams for refdes and method
streams = []
for method in methods: streams.append(common.list_streams(site, node, sensor, method))
streams

[['optaa_dj_dcl_instrument_recovered', 'optaa_dj_dcl_metadata_recovered'],
 ['optaa_dj_dcl_instrument', 'optaa_dj_dcl_metadata']]

In [5]:
# Show deployments for refdes (& get dates? How will this be used to build the data request?)
deployments = common.list_deployments(site, node, sensor)
deployments

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [6]:
# Also show annotations available from OOINet for this sensor
annotations = common.get_annotations(site, node, sensor)
annotations = pd.DataFrame(annotations)
with pd.option_context('display.max_colwidth', None):
  display(annotations["annotation"])

0                                                                                                                                                                                                                                                                                                                                                             Deployment 11: Telemeterd and recovered data potentially unavailable for several instruments on DCL27 from 2020-09-12 through 2020-09-14.
1                                                                                                                                                                                                                                                                                                                                                               The OPTAA burst duration was changed from 2 minutes to 4 minutes as of the start date/time of this annotation (start of Deployment 11).
2                       

In [8]:
# Pick method, stream, deployment from options shown above
method = methods[0]             # picking recovered host method; check OOINet for data availability to choose the most complete timeseries
stream = streams[0][0]          # data product or metadata; must pick a data stream from the list corresponding to the chosen method (first index)
deployment = deployments[13]    # picking a more recent deployment with less annotations indicating missing or suspect data
[sdt, edt] = common.get_deployment_dates(site, node, sensor, deployment)

# Build path to directory where we'll store data downloaded from OOI
filename = '-'.join([site, node, sensor, method, stream,f"deployment{deployment}"])+'.nc'
filepath = os.path.join(os.path.abspath('../data/external'), filename)

# Try importing locally saved data first
try:
    ds = xr.open_dataset(filepath)
except FileNotFoundError:
    # Otherwise download data with common.m2m_request and common.m2m_collect then write to disk
    data = common.m2m_request(site, node, sensor, method, stream, sdt, edt)
    ds = common.m2m_collect(data, use_dask=True)
    ds.to_netcdf(filepath)

Requesting:
	refdes: CP03ISSM-RID27-01-OPTAAD000
	method: recovered_host
	stream: optaa_dj_dcl_instrument_recovered
	from 2021-11-01T14:10:00.000Z to 2022-04-13T21:24:00.000Z
Waiting for OOINet to process and prepare data request, this may take up to 20 minutes.
Waiting: 100%|██████████| 400/400 [06:11<00:00,  1.08it/s]


C:\Users\kylene.cooley\Documents\GitHub\ooi-data-explorations\python\ooi_data_explorations\common.py:920: UserWarning: 2 of the 17 downloaded files failed to merge.
  warnings.warn(message)
c:\Users\kylene.cooley\AppData\Local\anaconda3\envs\optaa\Lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
c:\Users\kylene.cooley\AppData\Local\anaconda3\envs\optaa\Lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': 

In [9]:
ds

<xarray.Dataset>
Dimensions:                                       (time: 3588534, wavelength: 87)
Coordinates:
  * time                                          (time) datetime64[ns] 2021-...
  * wavelength                                    (wavelength) int32 0 1 ... 86
Data variables: (12/25)
    sea_water_temperature_qartod_results          (time) float64 dask.array<chunksize=(3588534,), meta=np.ndarray>
    deployment                                    (time) float64 dask.array<chunksize=(3588534,), meta=np.ndarray>
    sea_water_practical_salinity_qartod_executed  (time) object dask.array<chunksize=(3588534,), meta=np.ndarray>
    internal_timestamp                            (time) datetime64[ns] dask.array<chunksize=(3588534,), meta=np.ndarray>
    sea_water_temperature_qartod_executed         (time) object dask.array<chunksize=(3588534,), meta=np.ndarray>
    sea_water_practical_salinity                  (time) float64 dask.array<chunksize=(3588534,), meta=np.ndarray>
    ...                                            ...
    optical_absorption                            (time, wavelength) float64 dask.array<chunksize=(3505076, 87), meta=np.ndarray>
    c_signal_dark_counts                          (time) float64 dask.array<chunksize=(3505076,), meta=np.ndarray>
    pressure_counts                               (time) float64 dask.array<chunksize=(3505076,), meta=np.ndarray>
    a_signal_dark_counts                          (time) float64 dask.array<chunksize=(3505076,), meta=np.ndarray>
    c_reference_counts                            (time, wavelength) float64 dask.array<chunksize=(3505076, 87), meta=np.ndarray>
    a_reference_dark_counts                       (time) float64 dask.array<chunksize=(3505076,), meta=np.ndarray>
Attributes: (12/69)
    node:                               RID27
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_host
    stream:                             ctdbp_cdef_dcl_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.367167
    lon:                                -70.881817